# Preliminaries: model definition and utilities

In [ ]:
import math
import torch
import torch.nn as nn
import random
import numpy as np
import torch.nn.functional as F
import argparse
import os
import shutil
import time
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


In [ ]:
from neural_network import *
from utils import *
from metrics import *
from training_helpers import *
from datasets import *

In [ ]:
k = 5 # number of classes expert can predict
n_dataset = 10
Expert = synth_expert(k, n_dataset)


WideResNet from following [repo](https://github.com/xternalz/WideResNet-pytorch/blob/master/wideresnet.py)

# Active Learning Prep

In [ ]:
use_data_aug = False # whether to use data augmentation, no for now
n_dataset = 10  # cifar-10
normalize = transforms.Normalize(mean=[x / 255.0 for x in [125.3, 123.0, 113.9]],
                                    std=[x / 255.0 for x in [63.0, 62.1, 66.7]])

if use_data_aug:
    transform_train = transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda x: F.pad(x.unsqueeze(0),
                                            (4, 4, 4, 4), mode='reflect').squeeze()),
        transforms.ToPILImage(),
        transforms.RandomCrop(32),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ])
else:
    transform_train = transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    normalize
])

if n_dataset == 10:
    dataset = 'cifar10'
elif n_dataset == 100:
    dataset = 'cifar100'

kwargs = {'num_workers': 0, 'pin_memory': True}


train_dataset_all = datasets.__dict__[dataset.upper()]('../data', train=True, download=True,
                                                        transform=transform_train)
train_size = int(0.90 * len(train_dataset_all)) # validation set
test_size = len(train_dataset_all) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(train_dataset_all, [train_size, test_size])
#train_loader = torch.utils.data.DataLoader(train_dataset,
#                                           batch_size=128, shuffle=True, **kwargs)
#val_loader = torch.utils.data.DataLoader(val_dataset,
#                                            batch_size=128, shuffle=True, **kwargs)


normalize = transforms.Normalize(mean=[x / 255.0 for x in [125.3, 123.0, 113.9]],
                                 std=[x / 255.0 for x in [63.0, 62.1, 66.7]])
kwargs = {'num_workers': 1, 'pin_memory': True}

transform_test = transforms.Compose([
    transforms.ToTensor(),
    normalize
])
test_dataset = datasets.__dict__["cifar10".upper()]('../data', train=False, transform=transform_test, download=True)
#test_loader = torch.utils.data.DataLoader(
#    datasets.__dict__["cifar100".upper()]('../data', train=False, transform=transform_test, download=True),
#    batch_size=128, shuffle=True, **kwargs)



In [ ]:
dataset_train = CifarExpertDataset(np.array(train_dataset.dataset.data)[train_dataset.indices], np.array(train_dataset.dataset.targets)[train_dataset.indices], Expert.predict , [1]*len(train_dataset.indices))
dataset_val = CifarExpertDataset(np.array(val_dataset.dataset.data)[val_dataset.indices], np.array(val_dataset.dataset.targets)[val_dataset.indices], Expert.predict , [1]*len(val_dataset.indices))
dataset_test = CifarExpertDataset(test_dataset.data , test_dataset.targets, Expert.predict , [1]*len(test_dataset.targets))

dataLoaderTrain = DataLoader(dataset=dataset_train, batch_size=128, shuffle=True,  num_workers=0, pin_memory=True)
dataLoaderVal = DataLoader(dataset=dataset_val, batch_size=128, shuffle=False,  num_workers=0, pin_memory=True)
dataLoaderTest = DataLoader(dataset=dataset_test, batch_size=128, shuffle=False,  num_workers=0, pin_memory=True)

# Optimal Solution
Runs the L_{CE} method on the data with a simple neural network architecture

In [ ]:
model = NetSimple(n_dataset + 1, 100, 100, 1000,500).to(device)

In [ ]:
#run_reject(model, 10, Expert.predict, 70,0.5, dataLoaderTrain, dataLoaderVal)


# Active Learning
The way I'm currently doing things is:
- learn a classifier on all the data
- initiate active learning parameters
- run code for each active learning baseline and gather results
- plot results

In [ ]:
model = NetSimple(n_dataset + 1, 100, 100, 1000,500).to(device) # classifier model

In [ ]:
run_reject_class(model, 10, dataLoaderTrain, dataLoaderVal) # train classifier model

In [ ]:
metrics_print_classifier(model, dataLoaderVal) # show classifier results

In [ ]:
# Active Learning parameters
INITIAL_SIZE = 100 # initial size of data that has expert labels, randomly chosen
BATCH_SIZE_AL = 1000 # batch size for acquisition of expert labels
MAX_ROUNDS = 10 # how many rounds of expert queries: total size is initial_size + batch_size*max_rounds
EPOCH_TRAIN = 10 # how many epochs to train after each acquistion
MAX_TRIALS = 1 # repeat active learning for a number of trials to get error bars

In [ ]:
# initiate data for active learning with labeled and unlabeled set
all_indices = list(range(len(train_dataset.indices)))
all_data_x = np.array(train_dataset.dataset.data)[train_dataset.indices]
all_data_y = np.array(train_dataset.dataset.targets)[train_dataset.indices]

intial_random_set = random.sample(all_indices, INITIAL_SIZE)
indices_labeled  = intial_random_set
indices_unlabeled= list(set(all_indices) - set(indices_labeled))

dataset_train_labeled = CifarExpertDataset(all_data_x[indices_labeled], all_data_y[indices_labeled], Expert.predict , [1]*len(indices_labeled), indices_labeled)
dataset_train_unlabeled = CifarExpertDataset(all_data_x[indices_unlabeled], all_data_y[indices_unlabeled], Expert.predict , [0]*len(indices_unlabeled), indices_unlabeled)

dataLoaderTrainLabeled = DataLoader(dataset=dataset_train_labeled, batch_size=128, shuffle=True,  num_workers=0, pin_memory=True)
dataLoaderTrainUnlabeled = DataLoader(dataset=dataset_train_unlabeled, batch_size=128, shuffle=True,  num_workers=0, pin_memory=True)

In [ ]:
Intial_random_set = random.sample(all_indices, INITIAL_SIZE) # initial random set is constant

In [ ]:
# random selection
error_random_trials = []
for trial in range(MAX_TRIALS):
    # repeat for each trial
    print(f'\n \n \n Trial {trial} \n \n \n ')
    all_indices = list(range(len(train_dataset.indices)))
    all_data_x = np.array(train_dataset.dataset.data)[train_dataset.indices]
    all_data_y = np.array(train_dataset.dataset.targets)[train_dataset.indices]

    indices_labeled  = Intial_random_set
    indices_unlabeled= list(set(all_indices) - set(indices_labeled))

    dataset_train_labeled = CifarExpertDataset(all_data_x[indices_labeled], all_data_y[indices_labeled], Expert.predict , [1]*len(indices_labeled), indices_labeled)
    dataset_train_unlabeled = CifarExpertDataset(all_data_x[indices_unlabeled], all_data_y[indices_unlabeled], Expert.predict , [0]*len(indices_unlabeled), indices_unlabeled)

    dataLoaderTrainLabeled = DataLoader(dataset=dataset_train_labeled, batch_size=128, shuffle=True,  num_workers=0, pin_memory=True)
    dataLoaderTrainUnlabeled = DataLoader(dataset=dataset_train_unlabeled, batch_size=128, shuffle=True,  num_workers=0, pin_memory=True)

    # train expert model on initial labeled set
    model_expert = NetSimple(2, 100, 100, 1000,500).to(device)
    run_expert(model_expert, EPOCH_TRAIN, dataLoaderTrainLabeled, dataLoaderTrainLabeled)
    data_sizes = []
    error_random = []

    data_sizes.append(INITIAL_SIZE)
    metrics_random = metrics_print_2step(model, model_expert, Expert.predict, 10, dataLoaderVal)
    error_random.append(metrics_random['system accuracy'])


    for round in range(MAX_ROUNDS):
        # in each round of active learning
        # randomly sample data
        print(f'\n \n Round {round} \n \n')
        intial_random_set = random.sample(indices_unlabeled, BATCH_SIZE_AL) # sample random batch from unlabeled set
        # create new datasets with new labeled data
        indices_labeled  = indices_labeled + intial_random_set 
        indices_unlabeled= list(set(all_indices) - set(indices_labeled))
        dataset_train_labeled = CifarExpertDataset(all_data_x[indices_labeled], all_data_y[indices_labeled], Expert.predict , [1]*len(indices_labeled))
        dataset_train_unlabeled = CifarExpertDataset(all_data_x[indices_unlabeled], all_data_y[indices_unlabeled], Expert.predict , [0]*len(indices_unlabeled))
        dataLoaderTrainLabeled = DataLoader(dataset=dataset_train_labeled, batch_size=128, shuffle=True,  num_workers=0, pin_memory=True)
        dataLoaderTrainUnlabeled = DataLoader(dataset=dataset_train_unlabeled, batch_size=128, shuffle=True,  num_workers=0, pin_memory=True)
        dataLoaderTrainUnlabeledUnshuffled = DataLoader(dataset=dataset_train_unlabeled, batch_size=128, shuffle=False,  num_workers=0, pin_memory=True)

        # re-train expert model on new labeled data
        run_expert(model_expert, EPOCH_TRAIN, dataLoaderTrainLabeled, dataLoaderTrainLabeled)
        # get results of deferral and store them
        metrics_random = metrics_print_2step(model, model_expert, Expert.predict, 10, dataLoaderTest)
        error_random.append(metrics_random['system accuracy'])
        data_sizes.append((round+1)*BATCH_SIZE_AL + INITIAL_SIZE)
    error_random_trials.append(error_random)

In [ ]:
# confidence selection WITH LCE
# this is the exact algorithm in the paper
from scipy.stats import entropy

def get_least_confident_points(model, data_loader, budget):
    '''
    based on entropy score get points, can chagnge, but make sure to get max or min accordingly
    '''
    uncertainty_estimates = []
    indices_all = []
    for data in data_loader:
        images, labels, expert_preds, indices, _ = data
        images, labels, expert_preds = images.to(device), labels.to(device), expert_preds.to(device)
        outputs = model(images)
        batch_size = outputs.size()[0]  
        for i in range(0, batch_size):
            output_i =  outputs.data[i].cpu().numpy()
            entropy_i = entropy(output_i)
            #entropy_i = 1 - max(output_i)
            uncertainty_estimates.append(entropy_i)
            indices_all.append(indices[i].item())
    indices_all = np.array(indices_all)
    top_budget_indices = np.argsort(uncertainty_estimates)[-budget:]
    actual_indices = indices_all[top_budget_indices]
    uncertainty_estimates = np.array(uncertainty_estimates)
    return actual_indices
import copy
EPOCHS_DEFER = 10
error_confidence_trials_LCE = []


for trial in range(MAX_TRIALS):
    print(f'\n \n \n Trial {trial} \n \n \n ')
    # initialize data
    all_indices = list(range(len(train_dataset.indices)))
    all_data_x = np.array(train_dataset.dataset.data)[train_dataset.indices]
    all_data_y = np.array(train_dataset.dataset.targets)[train_dataset.indices]

    indices_labeled  = Intial_random_set
    indices_unlabeled= list(set(all_indices) - set(indices_labeled))

    dataset_train_labeled = CifarExpertDataset(all_data_x[indices_labeled], all_data_y[indices_labeled], Expert.predict , [1]*len(indices_labeled), indices_labeled)
    dataset_train_unlabeled = CifarExpertDataset(all_data_x[indices_unlabeled], all_data_y[indices_unlabeled], Expert.predict , [0]*len(indices_unlabeled), indices_unlabeled)

    dataLoaderTrainLabeled = DataLoader(dataset=dataset_train_labeled, batch_size=128, shuffle=True,  num_workers=0, pin_memory=True)
    dataLoaderTrainUnlabeled = DataLoader(dataset=dataset_train_unlabeled, batch_size=128, shuffle=True,  num_workers=0, pin_memory=True)

    # train expert model on labeled data
    model_expert = NetSimple(2, 100, 100, 1000,500).to(device)
    run_expert(model_expert, EPOCH_TRAIN, dataLoaderTrainLabeled, dataLoaderTrainLabeled) 

    data_sizes = []
    error_confidence = []
    data_sizes.append(INITIAL_SIZE)
    # train model to do classification
    model_lce = NetSimple(n_dataset + 1, 100, 100, 1000,500).to(device)
    run_reject_class(model_lce, EPOCH_TRAIN, dataLoaderTrain, dataLoaderVal)
    model_lce_saved = copy.deepcopy(model_lce.state_dict())
    
    # get expert model predictions on unlabeled data
    dataLoaderTrainUnlabeledUnshuffled = DataLoader(dataset=dataset_train_unlabeled, batch_size=128, shuffle=False,  num_workers=0, pin_memory=True)
    expert_preds_arr = []
    for data in dataLoaderTrainUnlabeledUnshuffled:
        images, labels, _, _, _ = data
        images = images.to(device)
        outputs_exp = model_expert(images)
        for i in range(outputs_exp.size()[0]):
            #pred_exp = np.argmax(outputs_exp.data[i].cpu().numpy())
            pred_exp = outputs_exp.data[i].cpu().numpy()
            pred_exp = pred_exp[1]
            expert_preds_arr.append(pred_exp)
    expert_preds_unlabeled = np.array(expert_preds_arr)
    expert_preds_labeled = np.array(Expert.predict (None, torch.FloatTensor(all_data_y[indices_labeled])))
    expert_preds_labeled = ( expert_preds_labeled == all_data_y[indices_labeled]) * 1
    expert_preds_combined = np.concatenate(( expert_preds_labeled, expert_preds_unlabeled))
    # create pseudo-labeled dataset
    dataset_train_pseudolabeled = CifarExpertDataset(np.concatenate((all_data_x[indices_labeled] , all_data_x[indices_unlabeled])),
                                                        np.concatenate((all_data_y[indices_labeled] , all_data_y[indices_unlabeled])), Expert.predict , [1]*(len(indices_labeled) + len(indices_unlabeled)), None,
                                                        expert_preds_combined)
    dataLoaderTrainPseudoLabeled = DataLoader(dataset=dataset_train_pseudolabeled, batch_size=128, shuffle=True,  num_workers=0, pin_memory=True)

    # train model on pseudo-labeled data
    run_reject_pseudo(model_lce, 10, Expert.predict, EPOCHS_DEFER, 1, dataLoaderTrainPseudoLabeled, dataLoaderTrainLabeled)
    metrics_confidence = metrics_print(model_lce, Expert.predict, n_dataset, dataLoaderTest)
    error_confidence.append(metrics_confidence['system accuracy'])
    for round in range(MAX_ROUNDS):
        model_lce.load_state_dict(model_lce_saved)
        # get points where expert model is least confident on
        print(f'\n \n Round {round} \n \n')
        indices_confidence =  random.sample(indices_unlabeled, BATCH_SIZE_AL)#get_least_confident_points(model_expert, dataLoaderTrainUnlabeled, BATCH_SIZE_AL)
        indices_labeled  = indices_labeled + list(indices_confidence) 
        indices_unlabeled= list(set(all_indices) - set(indices_labeled))
        dataset_train_labeled = CifarExpertDataset(all_data_x[indices_labeled], all_data_y[indices_labeled], Expert.predict , [1]*len(indices_labeled))
        dataset_train_unlabeled = CifarExpertDataset(all_data_x[indices_unlabeled], all_data_y[indices_unlabeled], Expert.predict , [0]*len(indices_unlabeled))
        dataLoaderTrainLabeled = DataLoader(dataset=dataset_train_labeled, batch_size=128, shuffle=True,  num_workers=0, pin_memory=True)
        dataLoaderTrainUnlabeled = DataLoader(dataset=dataset_train_unlabeled, batch_size=128, shuffle=True,  num_workers=0, pin_memory=True)
        # train model on labeled data
        run_expert(model_expert, EPOCH_TRAIN, dataLoaderTrainLabeled, dataLoaderTrainLabeled)
        # get expert predictions on unlabeled data
        dataLoaderTrainUnlabeledUnshuffled = DataLoader(dataset=dataset_train_unlabeled, batch_size=128, shuffle=False,  num_workers=0, pin_memory=True)
        expert_preds_arr = []
        for data in dataLoaderTrainUnlabeledUnshuffled:
            images, labels, _, _, _ = data
            images = images.to(device)
            outputs_exp = model_expert(images)
            for i in range(outputs_exp.size()[0]):
                #pred_exp = np.argmax(outputs_exp.data[i].cpu().numpy())
                pred_exp = outputs_exp.data[i].cpu().numpy()
                pred_exp = pred_exp[1]
                expert_preds_arr.append(pred_exp)
        expert_preds_unlabeled = np.array(expert_preds_arr)
        expert_preds_labeled = np.array(Expert.predict (None, torch.FloatTensor(all_data_y[indices_labeled])))
        expert_preds_labeled = ( expert_preds_labeled == all_data_y[indices_labeled]) * 1
        expert_preds_combined = np.concatenate(( expert_preds_labeled, expert_preds_unlabeled))
        # create pseudo-labeled dataset
        
        dataset_train_pseudolabeled = CifarExpertDataset(np.concatenate((all_data_x[indices_labeled] , all_data_x[indices_unlabeled])),
                                                         np.concatenate((all_data_y[indices_labeled] , all_data_y[indices_unlabeled])), Expert.predict , [1]*(len(indices_labeled) + len(indices_unlabeled))  , None,
                                                         expert_preds_combined)
        dataLoaderTrainPseudoLabeled = DataLoader(dataset=dataset_train_pseudolabeled, batch_size=128, shuffle=True,  num_workers=0, pin_memory=True)

        # train model on pseudo labeled data
        best_score = 0
        best_model = None
        for alpha in [1]:
            print(f'alpha {alpha}')
            model_lce.load_state_dict(model_lce_saved)
            model_dict_alpha = run_reject_pseudo(model_lce, 10, Expert.predict, EPOCHS_DEFER, 1, dataLoaderTrainPseudoLabeled, dataLoaderTest, True, EPOCHS_DEFER-1)
            model_lce.load_state_dict(model_dict_alpha)
            score = metrics_print(model_lce, Expert.predict, n_dataset, dataLoaderTest)['system accuracy']
            if score >= best_score:
                best_score =  score
                best_model = model_dict_alpha
        model_lce.load_state_dict(best_model)

        #run_reject(model_lce, 10, Expert.predict, EPOCHS_DEFER, 1, dataLoaderTrainPseudoLabeled, dataLoaderTrainLabeled)
        metrics_confidence = metrics_print(model_lce, Expert.predict, n_dataset, dataLoaderTest)
        error_confidence.append(metrics_confidence['system accuracy'])
        data_sizes.append((round+1)*BATCH_SIZE_AL + INITIAL_SIZE)
    error_confidence_trials_LCE.append(error_confidence)

# plot

In [ ]:
#!apt install texlive-fonts-recommended texlive-fonts-extra cm-super dvipng
import matplotlib

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rc('text', usetex=False)
plt.rc('font', family='serif')
def get_conf_interval(arr):
    alpha_level = 0.4
    err  = st.t.interval(alpha_level, len(arr)-1, loc=np.mean(arr), scale=st.sem(arr))[1]/2  - st.t.interval(alpha_level, len(arr)-1, loc=np.mean(arr), scale=st.sem(arr))[0]/2 
    return err

In [ ]:
# uncomment things to plot more

#avgs_rand = [np.average([scores_oracle[triall] - scores_lime[triall][i] for triall in range(actual_max_trials)]) for i in range(len(teaching_sizes))]
#stds_rand = [np.std([scores_oracle[triall] -scores_lime[triall][i] for triall in range(actual_max_trials)]) for i in range(len(teaching_sizes))]
#plt.errorbar(list(range(1,len(teaching_sizes)+1)),  avgs_rand, yerr=stds_rand, label=f'random')
avgs_rand = [np.average([error_random_trials[triall][i]  for triall in range(MAX_TRIALS)]) for i in range(MAX_ROUNDS + 1)]
stds_rand = [np.std([error_random_trials[triall][i]  for triall in range(MAX_TRIALS)]) for i in range(MAX_ROUNDS + 1)]
plt.errorbar(data_sizes,  avgs_rand, yerr=stds_rand, marker = "+",  label=f'Random')
'''
avgs_rand = [np.average([error_confidence_rejector_trials[triall][i]  for triall in range(MAX_TRIALS)]) for i in range(MAX_ROUNDS + 1)]
stds_rand = [np.std([error_confidence_rejector_trials[triall][i]  for triall in range(MAX_TRIALS)]) for i in range(MAX_ROUNDS + 1)]
plt.errorbar(data_sizes,  avgs_rand, yerr=stds_rand, marker = "+",  label=f'Rejector')

avgs_rand = [np.average([error_confidence_trials[triall][i]  for triall in range(MAX_TRIALS)]) for i in range(MAX_ROUNDS+1)]
stds_rand = [np.std([error_confidence_trials[triall][i]  for triall in range(MAX_TRIALS)]) for i in range(MAX_ROUNDS+1)]
plt.errorbar(data_sizes,  avgs_rand, yerr=stds_rand, marker = "o",  label=f'Entropy Sampling')
'''
'''
avgs_rand = [np.average([error_confidence_trials_LCE[triall][i]  for triall in range(MAX_TRIALS)]) for i in range(MAX_ROUNDS+1)]
stds_rand = [np.std([error_confidence_trials_LCE[triall][i]  for triall in range(MAX_TRIALS)]) for i in range(MAX_ROUNDS+1)]
plt.errorbar(data_sizes,  avgs_rand, yerr=stds_rand, marker = "o",  label=f'Entropy Sampling (Ensemble)')
'''

#plt.errorbar(data_sizes,  error_confidence_rejector, yerr=[0]*len(error_confidence_rejector), marker = "+",  label=f'Rejector Uncertainty')
#plt.errorbar(data_sizes,  errors_LCE, yerr=[0]*len(errors_LCE), marker = "o",  label=f'LCE random')
#plt.errorbar(data_sizes,  errors_teaching, yerr=[0]*len(error_confidence_rejector), marker = "o",  label=f'Teaching')


ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()   
plt.grid()
plt.legend(fontsize='large')
plt.legend()
plt.ylabel('System Accuracy',  fontsize='x-large')
plt.xlabel('Acquired data size', fontsize='x-large')
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 6
fig_size[1] = 4
#plt.savefig("teaching_complexity_cifar10.pdf", dpi = 1000)
plt.show()

# Linear Representations
This is the same as above but with linear representation and linear networks instead of training on images

In [ ]:
# Active Learning parameters
INITIAL_SIZE = 100
BATCH_SIZE_AL = 1000
MAX_ROUNDS = 10
EPOCH_TRAIN = 10
MAX_TRIALS = 1

In [ ]:
dataset_train = CifarExpertDataset(np.array(train_dataset.dataset.data)[train_dataset.indices], np.array(train_dataset.dataset.targets)[train_dataset.indices], Expert.predict , [1]*len(train_dataset.indices))
dataset_val = CifarExpertDataset(np.array(val_dataset.dataset.data)[val_dataset.indices], np.array(val_dataset.dataset.targets)[val_dataset.indices], Expert.predict , [1]*len(val_dataset.indices))
dataset_test = CifarExpertDataset(test_dataset.data , test_dataset.targets, Expert.predict , [1]*len(test_dataset.targets))

dataLoaderTrain = DataLoader(dataset=dataset_train, batch_size=128, shuffle=True,  num_workers=0, pin_memory=True)
dataLoaderVal = DataLoader(dataset=dataset_val, batch_size=128, shuffle=False,  num_workers=0, pin_memory=True)
dataLoaderTest = DataLoader(dataset=dataset_test, batch_size=128, shuffle=False,  num_workers=0, pin_memory=True)

In [ ]:
hidden_repr_dim = 50
model = NetSimple(n_dataset + 1, 100, 100, 1000, hidden_repr_dim).to(device)
run_reject_class(model, 15, dataLoaderTrain, dataLoaderVal)

In [ ]:
metrics_print_classifier(model, dataLoaderVal)

In [ ]:
# re-initialize datasets with linear representations
dataset_train = CifarExpertDatasetLinear(np.array(train_dataset.dataset.data)[train_dataset.indices], np.array(train_dataset.dataset.targets)[train_dataset.indices], Expert.predict , [1]*len(train_dataset.indices),None, model)
dataset_val = CifarExpertDatasetLinear(np.array(val_dataset.dataset.data)[val_dataset.indices], np.array(val_dataset.dataset.targets)[val_dataset.indices], Expert.predict , [1]*len(val_dataset.indices),None, model)
dataset_test = CifarExpertDatasetLinear(test_dataset.data , test_dataset.targets, Expert.predict , [1]*len(test_dataset.targets),None,  model)

dataLoaderTrain = DataLoader(dataset=dataset_train, batch_size=128, shuffle=False,  num_workers=0, pin_memory=True)
dataLoaderVal = DataLoader(dataset=dataset_val, batch_size=128, shuffle=False,  num_workers=0, pin_memory=True)
dataLoaderTest = DataLoader(dataset=dataset_test, batch_size=128, shuffle=False,  num_workers=0, pin_memory=True)

In [ ]:
all_indices = list(range(len(train_dataset.indices)))
all_data_x = np.array(train_dataset.dataset.data)[train_dataset.indices]
all_data_y = np.array(train_dataset.dataset.targets)[train_dataset.indices]

intial_random_set = random.sample(all_indices, INITIAL_SIZE)
indices_labeled  = intial_random_set
indices_unlabeled= list(set(all_indices) - set(indices_labeled))



dataset_train_labeled = CifarExpertDatasetLinear(all_data_x[indices_labeled], all_data_y[indices_labeled], Expert.predict , [1]*len(indices_labeled), indices_labeled, model)
dataset_train_unlabeled = CifarExpertDatasetLinear(all_data_x[indices_unlabeled], all_data_y[indices_unlabeled], Expert.predict , [0]*len(indices_unlabeled), indices_unlabeled, model)

dataLoaderTrainLabeled = DataLoader(dataset=dataset_train_labeled, batch_size=128, shuffle=True,  num_workers=0, pin_memory=True)
dataLoaderTrainUnlabeled = DataLoader(dataset=dataset_train_unlabeled, batch_size=128, shuffle=True,  num_workers=0, pin_memory=True)

The below are re-intialized to work with linear representations
Make sure to not run this beforee the above, because it re-defines methods imported

In [ ]:
def train_expert_confidence(train_loader, model, optimizer, scheduler, epoch):
    """Train for one epoch on the training set without deferral"""
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, label, expert_pred, _, _ ) in enumerate(train_loader):
        expert_pred = expert_pred.long()
        expert_pred = (expert_pred == label) *1
        target = expert_pred.to(device)
        input = input.to(device)
        # compute output
        output = model(input)

        # compute loss
        loss = my_CrossEntropyLoss(output, target)

        # measure accuracy and record loss
        prec1 = accuracy(output.data, target, topk=(1,))[0]
        losses.update(loss.data.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 10 == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                epoch, i, len(train_loader), batch_time=batch_time,
                loss=losses, top1=top1))
            


def metrics_print_expert(model, data_loader, defer_net = False):
    '''
    model: model
    data_loader: data loader
    '''
    correct = 0
    total = 0
    # again no gradients needed
    with torch.no_grad():
        for data in data_loader:
            images, label, expert_pred, _ ,_ = data
            expert_pred = expert_pred.long()
            expert_pred = (expert_pred == label) *1
            images, labels = images.to(device), expert_pred.to(device)
            outputs = model(images)
            _, predictions = torch.max(outputs.data, 1) # maybe no .data

            total += labels.size(0)
            correct += (predictions == labels).sum().item()

    print('Accuracy of the network on the %d test images: %.3f %%' % (total,
        100 * correct / total))

def run_expert(model, epochs, train_loader, val_loader):
    '''
    only train classifier
    model: WideResNet model
    epochs: number of epochs to train
    '''
    # get the number of model parameters
    print('Number of model parameters: {}'.format(
        sum([p.data.nelement() for p in model.parameters()])))

    # define loss function (criterion) and optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False)


    # cosine learning rate
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(train_loader) * epochs)

    for epoch in range(0, epochs):
        # train for one epoch
        train_expert_confidence(train_loader, model, optimizer, scheduler, epoch)
        if epoch % 10 == 0:
            metrics_print_expert(model, val_loader)
    metrics_print_expert(model, val_loader)

In [ ]:
def metrics_print_2step(net_mod, net_exp, expert_fn, n_classes, loader):
    correct = 0
    correct_sys = 0
    exp = 0
    exp_total = 0
    total = 0
    real_total = 0
    with torch.no_grad():
        for data in loader:
            images, labels, expert_preds, _, images_orig = data
            images, labels, expert_preds, images_orig = images.to(device), labels.to(device), expert_preds.to(device), images_orig.to(device)
            outputs_mod = net_mod(images_orig)
            outputs_exp = net_exp(images)
            _, predicted = torch.max(outputs_mod.data, 1)
            _, predicted_exp = torch.max(outputs_exp.data, 1)
            batch_size = outputs_mod.size()[0]  # batch_size
            exp_prediction = expert_fn(images, labels)
            for i in range(0, batch_size):
                r_score =  outputs_mod.data[i][predicted[i].item()].item()
                r_score = outputs_exp.data[i][1].item() - r_score
                r = 0
                if r_score >= 0:
                    r = 1
                else:
                    r = 0
                if r == 0:
                    total += 1
                    correct += (predicted[i] == labels[i]).item()
                    correct_sys += (predicted[i] == labels[i]).item()
                if r == 1:
                    exp += (exp_prediction[i] == labels[i].item())
                    correct_sys += (exp_prediction[i] == labels[i].item())
                    exp_total += 1
                real_total += 1
    cov = str(total) + str(" out of") + str(real_total)
    to_print = {"coverage": cov, "system accuracy": 100 * correct_sys / real_total,
                "expert accuracy": 100 * exp / (exp_total + 0.0002),
                "classifier accuracy": 100 * correct / (total + 0.0001)}
    return to_print
    print(to_print)

In [ ]:
class Linear_net(nn.Module):
    '''
    Linear multiclass classifier with unit init
    '''
    def __init__(self, input_dim, out_dim):
        super(Linear_net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(input_dim, out_dim)
        torch.nn.init.ones_(self.fc1.weight)
        self.softmax = nn.Softmax()

    def forward(self, x):
        x = self.fc1(x)
        x = self.softmax(x)
        return x


Need to re-write this to add MAX_TRIALS (run this multiple times)

In [ ]:
# random selection
Intial_random_set = random.sample(all_indices, INITIAL_SIZE)

all_indices = list(range(len(train_dataset.indices)))
all_data_x = np.array(train_dataset.dataset.data)[train_dataset.indices]
all_data_y = np.array(train_dataset.dataset.targets)[train_dataset.indices]

indices_labeled  = Intial_random_set
indices_unlabeled= list(set(all_indices) - set(indices_labeled))

dataset_train_labeled = CifarExpertDatasetLinear(all_data_x[indices_labeled], all_data_y[indices_labeled], Expert.predict , [1]*len(indices_labeled), indices_labeled, model)
dataset_train_unlabeled = CifarExpertDatasetLinear(all_data_x[indices_unlabeled], all_data_y[indices_unlabeled], Expert.predict , [0]*len(indices_unlabeled), indices_unlabeled, model)

dataLoaderTrainLabeled = DataLoader(dataset=dataset_train_labeled, batch_size=1000, shuffle=True,  num_workers=0, pin_memory=True)
dataLoaderTrainUnlabeled = DataLoader(dataset=dataset_train_unlabeled, batch_size=1000, shuffle=True,  num_workers=0, pin_memory=True)


model_expert = Linear_net(hidden_repr_dim, 2).to(device)
run_expert(model_expert, 10, dataLoaderTrainLabeled, dataLoaderTrainLabeled)
data_sizes = []
error_random = []
data_sizes.append(INITIAL_SIZE)
metrics_random = metrics_print_2step(model, model_expert, Expert.predict, 10, dataLoaderVal)
error_random.append(metrics_random['system accuracy'])
for round in range(MAX_ROUNDS):
    print(f'\n \n Round {round} \n \n')
    intial_random_set = random.sample(indices_unlabeled, BATCH_SIZE_AL)

    indices_labeled  = indices_labeled + intial_random_set 
    indices_unlabeled= list(set(all_indices) - set(indices_labeled))
    dataset_train_labeled = CifarExpertDatasetLinear(all_data_x[indices_labeled], all_data_y[indices_labeled], Expert.predict , [1]*len(indices_labeled), None, model)
    dataset_train_unlabeled = CifarExpertDatasetLinear(all_data_x[indices_unlabeled], all_data_y[indices_unlabeled], Expert.predict , [0]*len(indices_unlabeled), None, model)
    dataLoaderTrainLabeled = DataLoader(dataset=dataset_train_labeled, batch_size=1000, shuffle=True,  num_workers=0, pin_memory=True)
    dataLoaderTrainUnlabeled = DataLoader(dataset=dataset_train_unlabeled, batch_size=1000, shuffle=True,  num_workers=0, pin_memory=True)
    model_expert = Linear_net(hidden_repr_dim, 2).to(device)

    run_expert(model_expert, 10, dataLoaderTrainLabeled, dataLoaderTrainLabeled)

    metrics_random = metrics_print_2step(model, model_expert, Expert.predict, 10, dataLoaderVal)
    error_random.append(metrics_random['system accuracy'])
    data_sizes.append((round+1)*BATCH_SIZE_AL + INITIAL_SIZE)

In [ ]:
# confidence selection
from scipy.stats import entropy

def get_least_confident_points(model, data_loader, budget):
    '''
    based on entropy score, can chagnge, but make sure to get max or min accordingly
    '''
    uncertainty_estimates = []
    indices_all = []
    for data in data_loader:
        images, labels, expert_preds, indices, _ = data
        images, labels, expert_preds = images.to(device), labels.to(device), expert_preds.to(device)
        outputs = model(images)
        batch_size = outputs.size()[0]  
        for i in range(0, batch_size):
            output_i =  outputs.data[i].cpu().numpy()
            entropy_i = entropy(output_i)
            #entropy_i = 1 - max(output_i)
            uncertainty_estimates.append(entropy_i)
            indices_all.append(indices[i].item())
    indices_all = np.array(indices_all)
    top_budget_indices = np.argsort(uncertainty_estimates)[-budget:]
    actual_indices = indices_all[top_budget_indices]
    uncertainty_estimates = np.array(uncertainty_estimates)
    return actual_indices
all_indices = list(range(len(train_dataset.indices)))
all_data_x = np.array(train_dataset.dataset.data)[train_dataset.indices]
all_data_y = np.array(train_dataset.dataset.targets)[train_dataset.indices]

indices_labeled  = Intial_random_set
indices_unlabeled= list(set(all_indices) - set(indices_labeled))

dataset_train_labeled = CifarExpertDatasetLinear(all_data_x[indices_labeled], all_data_y[indices_labeled], Expert.predict , [1]*len(indices_labeled), None, model)
dataset_train_unlabeled = CifarExpertDatasetLinear(all_data_x[indices_unlabeled], all_data_y[indices_unlabeled], Expert.predict , [0]*len(indices_unlabeled), None, model)
    
dataLoaderTrainLabeled = DataLoader(dataset=dataset_train_labeled, batch_size=1000, shuffle=True,  num_workers=0, pin_memory=True)
dataLoaderTrainUnlabeled = DataLoader(dataset=dataset_train_unlabeled, batch_size=1000, shuffle=True,  num_workers=0, pin_memory=True)


model_expert = Linear_net(hidden_repr_dim, 2).to(device)
run_expert(model_expert, 10, dataLoaderTrainLabeled, dataLoaderTrainLabeled)
data_sizes = []
error_confidence = []
data_sizes.append(INITIAL_SIZE)
metrics_confidence = metrics_print_2step(model, model_expert, Expert.predict, 10, dataLoaderVal)
error_confidence.append(metrics_confidence['system accuracy'])


for round in range(MAX_ROUNDS):
    print(f'\n \n Round {round} \n \n')
    indices_confidence = get_least_confident_points(model_expert, dataLoaderTrainUnlabeled, BATCH_SIZE_AL)

    indices_labeled  = indices_labeled + intial_random_set 
    indices_unlabeled= list(set(all_indices) - set(indices_labeled))
    dataset_train_labeled = CifarExpertDatasetLinear(all_data_x[indices_labeled], all_data_y[indices_labeled], Expert.predict , [1]*len(indices_labeled), None, model)
    dataset_train_unlabeled = CifarExpertDatasetLinear(all_data_x[indices_unlabeled], all_data_y[indices_unlabeled], Expert.predict , [0]*len(indices_unlabeled), None, model)
    dataLoaderTrainLabeled = DataLoader(dataset=dataset_train_labeled, batch_size=1000, shuffle=True,  num_workers=0, pin_memory=True)
    dataLoaderTrainUnlabeled = DataLoader(dataset=dataset_train_unlabeled, batch_size=1000, shuffle=True,  num_workers=0, pin_memory=True)
    
    model_expert = Linear_net(hidden_repr_dim, 2).to(device)

    run_expert(model_expert, 50, dataLoaderTrainLabeled, dataLoaderTrainLabeled)

    metrics_confidence = metrics_print_2step(model, model_expert, Expert.predict, 10, dataLoaderVal)
    error_confidence.append(metrics_confidence['system accuracy'])
    data_sizes.append((round+1)*BATCH_SIZE_AL + INITIAL_SIZE)

In [ ]:

#avgs_rand = [np.average([scores_oracle[triall] - scores_lime[triall][i] for triall in range(actual_max_trials)]) for i in range(len(teaching_sizes))]
#stds_rand = [np.std([scores_oracle[triall] -scores_lime[triall][i] for triall in range(actual_max_trials)]) for i in range(len(teaching_sizes))]
#plt.errorbar(list(range(1,len(teaching_sizes)+1)),  avgs_rand, yerr=stds_rand, label=f'random')
plt.errorbar(data_sizes,  error_random, yerr=[0]*len(data_sizes), marker = "+",  label=f'Random')
#plt.errorbar(data_sizes2,  error_confidence[:59], yerr=[0]*len(data_sizes2), marker = "+",  label=f'Entropy Sampling')
#plt.errorbar(data_sizes,  error_confidence_rejector, yerr=[0]*len(error_confidence_rejector), marker = "+",  label=f'Rejector Uncertainty')
#plt.errorbar(data_sizes,  errors_LCE, yerr=[0]*len(errors_LCE), marker = "o",  label=f'LCE random')
#plt.errorbar(data_sizes,  errors_teaching, yerr=[0]*len(error_confidence_rejector), marker = "o",  label=f'Teaching')


ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()   
plt.grid()
plt.legend(fontsize='large')
plt.legend()
plt.ylabel('System Accuracy',  fontsize='x-large')
plt.xlabel('Acquired data size', fontsize='x-large')
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 6
fig_size[1] = 4
#plt.savefig("teaching_complexity_cifar10.pdf", dpi = 1000)
plt.show()